In [1]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split

In [2]:
pd.set_option('display.max_rows', 10000)
pd.set_option('display.max_columns', 500)
pd.set_option('display.precision',150)

In [3]:
df = pd.read_json('../data/whats-cooking/train.json')

In [4]:
def lower_list(arr):
    return [x.lower() for x in arr]

In [5]:
df['ingredient_count'] = df['ingredients'].map(lambda x: len(x))

In [6]:
df['ingredients'] = df['ingredients'].map(lower_list)

In [7]:
df.head(10)

,id,cuisine,ingredients,ingredient_count
0,10259,greek,"[romaine lettuce, black olives, grape tomatoes...",9
1,25693,southern_us,"[plain flour, ground pepper, salt, tomatoes, g...",11
2,20130,filipino,"[eggs, pepper, salt, mayonaise, cooking oil, g...",12
3,22213,indian,"[water, vegetable oil, wheat, salt]",4
4,13162,indian,"[black pepper, shallots, cornflour, cayenne pe...",20
5,6602,jamaican,"[plain flour, sugar, butter, eggs, fresh ginge...",12
6,42779,spanish,"[olive oil, salt, medium shrimp, pepper, garli...",13
7,3735,italian,"[sugar, pistachio nuts, white almond bark, flo...",10
8,16903,mexican,"[olive oil, purple onion, fresh pineapple, por...",13
9,12734,italian,"[chopped tomatoes, fresh basil, garlic, extra-...",6


In [8]:
def remove_spaces(arr):
    return [x.replace(' ', '_') for x in arr]

In [9]:
def strip_words(arr):
     return [x.strip() for x in arr]

In [10]:
def remove_commas(arr):
    return [x.replace(',', '') for x in arr]

In [11]:
def remove_dots(arr):
    return [x.replace('.', '') for x in arr]

In [12]:
def remove_weird_characters(arr):
    weird_characters = [',', '.', '(', ')', "'", 'u"\u2122"']
    for char in weird_characters:
        for i in range(len(arr)):
            arr[i].replace(char, '')
    return arr

In [13]:

df['ingredients'] = df['ingredients'].map(strip_words)
df['ingredients'] = df['ingredients'].map(remove_weird_characters)
df['ingredients'] = df['ingredients'].map(remove_spaces)


In [14]:
df['ingredients'].head(15)

0     [romaine_lettuce, black_olives, grape_tomatoes...
1     [plain_flour, ground_pepper, salt, tomatoes, g...
2     [eggs, pepper, salt, mayonaise, cooking_oil, g...
3                   [water, vegetable_oil, wheat, salt]
4     [black_pepper, shallots, cornflour, cayenne_pe...
5     [plain_flour, sugar, butter, eggs, fresh_ginge...
6     [olive_oil, salt, medium_shrimp, pepper, garli...
7     [sugar, pistachio_nuts, white_almond_bark, flo...
8     [olive_oil, purple_onion, fresh_pineapple, por...
9     [chopped_tomatoes, fresh_basil, garlic, extra-...
10    [pimentos, sweet_pepper, dried_oregano, olive_...
11    [low_sodium_soy_sauce, fresh_ginger, dry_musta...
12    [italian_parsley_leaves, walnuts, hot_red_pepp...
13    [ground_cinnamon, fresh_cilantro, chili_powder...
14    [fresh_parmesan_cheese, butter, all-purpose_fl...
Name: ingredients, dtype: object

In [15]:
df['ingredients'] = df['ingredients'].map(lambda x: " ".join(x)) 

In [16]:
df['ingredients'] = df['ingredients'].replace('[^a-zA-Z\d\s:]', '', regex=True)

In [17]:
pd.set_option('max_colwidth', 300)

In [18]:
print(df[df['ingredients'].str.contains('or_best_food_real_mayonnais')]['ingredients'])

Series([], Name: ingredients, dtype: object)


In [19]:
print(df[df['ingredients'].str.contains(',')]['ingredients'])

Series([], Name: ingredients, dtype: object)


In [20]:
count = CountVectorizer(min_df=2)

In [21]:
X = df['ingredients']
y = df['cuisine']

In [22]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42, stratify=y)

In [23]:
X_train_cv = count.fit_transform(X_train)
X_test_cv = count.transform(X_test)
df_X_train = pd.DataFrame(X_train_cv.todense(), columns=count.get_feature_names())

In [24]:
df_X_train['koshersalt']

0        0
1        0
2        0
3        0
4        0
        ..
29825    0
29826    0
29827    0
29828    0
29829    1
Name: koshersalt, Length: 29830, dtype: int64

In [25]:
df_X_train.head()

,10ozfrozenchoppedspinachthawedandsqueezeddry,145ozdicedtomatoes,14ozsweetenedcondensedmilk,15ozrefriedbeans,1lowfatbuttermilk,1lowfatcottagecheese,1lowfatmilk,212to3lbchickencutintoservingpieces,25lesssodiumchickenbroth,2lowfatcottagecheese,2reducedfatmilk,33lesssodiumham,40lesssodiumtacoseasoning,40lesssodiumtacoseasoningmix,7up,95leangroundbeef,aai,aaipowder,abalone,aburaage,accentseasoning,achiote,achiotepaste,achiotepowder,acinidipepe,ackee,acornsquash,activedryyeast,adobo,adobosauce,adoboseasoning,adzukibeans,agar,agavenectar,agedbalsamicvinegar,agedcheddarcheese,ahi,ahitunasteaks,aioli,ajinomoto,ajwain,akamiso,alaskankingcrablegs,albacoretunainwater,alcohol,ale,aleppopepper,alfredosauce,allbeefhotdogs,allpotatopurpos,allpurposeflour,allpurposeseasoning,allpurposeunbleachedflour,allspice,allspiceberries,almondbutter,almondextract,almondflour,almondliqueur,almondmeal,almondmilk,almondpaste,almonds,alphabetpasta,amaranth,amaretti,amaretticookies,amaretto,amarettoliqueur,amchur,americancheese,americancheeseslices,ampalaya,anaheimchile,anasazibeans,ancho,anchochilepepper,anchochiligroundpepper,anchopowder,anchovies,anchovyfilets,anchovyfillets,anchovypaste,andfatfreehalfhalf,andouillechickensausage,andouillesausage,andouillesausagelinks,angelfoodcake,angelfoodcakemix,angelhair,angosturabitters,anise,aniseextract,aniseoil,anisepowder,aniseseed,anisette,anjoupears,annatto,annattoseeds,applebrandy,applebutter,applecider,applecidervinegar,applejelly,applejuice,applejuiceconcentrate,applepiefilling,applepiespice,apples,applesauce,applewoodsmokedbacon,apricothalves,apricotjam,apricotnectar,apricotpreserves,apricots,arbolchile,arboriorice,armagnac,arrowroot,arrowrootpowder,artichokebottoms,artichokehearts,artichokes,artichokheartmarin,arugula,asadero,asafetida,asafetidapowder,asafoetida,asafoetidapowder,asiago,asianbasil,asianchilepaste,asianchiliredsauc,asianchilisauce,asianeggplants,asianfishsauce,asiannoodles,asianpear,asianricenoodles,asianwheatnoodles,asparagus,asparagusspears,asparagustips,aspic,assortedfreshvegetables,atta,aujusgravymix,avocado,avocadoleaves,avocadooil,babyartichokes,babyarugula,babybackribs,babybokchoy,babybroccoli,babycarrots,babycorn,babyeggplants,babygemlettuce,babygreens,babyleaflettuce,babylimabeans,babyportobellomushrooms,babypotatoes,babyradishes,babyspinach,babyspinachleaves,babytatsoi,babyturnips,babyzucchini,bacardi,backbacon,backbaconrashers,bacon,baconbits,bacondrippings,baconfat,bacongrease,baconpieces,baconsalt,baconslices,bagels,baguette,baileysirishcreamliqueur,bakedbeans,bakedham,bakedpizzacrust,bakedtortillachips,bakingapples,bakingchocolate,bakingmix,bakingpotatoes,bakingpowder,bakingsoda,bakingspray,bakingyeast,balm,balsamicobianco,balsamicreduction,balsamicvinaigrette,balsamicvinaigrettesaladdressing,balsamicvinegar,bambooshoots,bananablossom,bananaleaves,bananaliqueur,bananapeppers,bananas,bananasquash,banger,barbecuedpork,barbecuerub,barbecuesauce,barilla,barley,barleyflour,barleymiso,bartlettpears,base,basil,basildriedleaves,basilleaves,basilmayonnaise,basilpestosauce,basmati,basmatirice,bassfillets,baton,batter,bawanggoreng,bayleaf,bayleaves,bayscallops,bbqsauce,bbqseasoning,beancurd,beancurdskins,beandip,beanpaste,beans,beansauce,beansoup,beansprouts,beanthreads,beanthreadvermicelli,beateneggs,beaujolais,...,vealchops,vealcutlets,vealdemiglace,vealforstew,vealloinchops,vealribchops,vealscallops,vealshanks,vealshoulder,vealstock,veganbutter,veganmargarine,veganmayonnaise,veganparmesancheese,vegansourcream,veganworcestershiresauce,vegetablebouillon,vegetablebouilloncube,vegetablebroth,vegetabledemiglace,vegetablegumbo,vegetablejuice,vegetablejuicecocktail,vegetableoil,vegetableoilcookingspray,vegetableoilspray,vegetables,vegetableseasoning,vegetableshortening,vegetablestock,vegetarianoystersauce,vegetarianrefriedbeans,veggiecrumbles,veggies,velveeta,venison,verjus,vermicelli,vermicellinoodles,vermouth,vidalia,vidaliaonion,vietnamesecoriander,vietnamesefishsauce,vietnamesericepaper,vi